In [2]:
#@title Get text train and test ,X and Y
nrows_train = 20000 # @param {type:"integer"}
sequence_length = 13 # @param {type:"integer"}

feed_lenght = 8

import pandas as pd
import numpy as np
import time
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
import json
from keras.preprocessing.text import tokenizer_from_json

#load fusion
fus = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/airport_ml/LEVX/notebooks/LEVXfusionml.csv",
                  parse_dates=["time"], index_col="time")

#Get the train and test train
texts_train = fus["fusion"].sample(nrows_train,)
texts_test = fus["fusion"].drop(texts_train.index)

#save texts test
texts_test.to_csv("/content/drive/MyDrive/Colab Notebooks/airport_ml/LEVX/notebooks/LEVXtexts_testml.csv")

# Tokenize text
tokenizer = Tokenizer()

#tokenizer.fit_on_texts(texts_train)
tokenizer.fit_on_texts(fus["fusion"])

#Save tokenizer
tokenizer_json = tokenizer.to_json()

# Save the JSON configuration to a file
with open('/content/drive/MyDrive/Colab Notebooks/airport_ml/LEVX/algorithms/LEVXtokenizerml.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(tokenizer_json, ensure_ascii=False))

sequences = tokenizer.texts_to_sequences(texts_train)

# Prepare input and output data
X = []
y = []
for sequence in sequences:
    for i in range(1, len(sequence)):
        x_seq = sequence[:i]
        x_seq_padded = pad_sequences([x_seq], maxlen=sequence_length, padding='pre')
        X.append(x_seq_padded[0])
        y.append(sequence[i])

X = np.array(X)
y = np.array(y)

#filter seed/model words
df = pd.DataFrame(X)
df["y"] = y
df_fil =  df[(df.iloc[:, :sequence_length-feed_lenght+1] != 0).any(axis=1)]
X = df_fil.iloc[:, :-1].values
y = df_fil.iloc[:, -1].values

# One hot encode the outputs
y = np.eye(len(tokenizer.word_index) + 1)[y]

print("Dictionary size (words): len(y[i]):",len(y[1]) )
print("Total X variables:",len(X) )

print("\nText train example1")
display(texts_train[0])

print("\nSequences example 1")
display(sequences[0])

print("\nText train example2")
display(texts_train[1])

print("\nSequences example 2")
display(sequences[1])

print("\nX")
display(X[:25])

print("\nDictionary first 10 words")
display({k: tokenizer.word_index[k] for k in list(tokenizer.word_index)[:10]})


Dictionary size (words): len(y[i]): 3033
Total X variables: 176733

Text train example1


'01013kt 9999 WM M040 M 26 16 q1015 01010KT 330V070 CAVOK 26/15 Q1014 NOSIG'


Sequences example 1


[253, 1, 3, 292, 4, 109, 10, 31, 684, 513, 5, 109, 12, 41, 2]


Text train example2


...

[171, 1, 3, 78, 4, 17, 11, 30, 91, 5, 10, 12, 30, 2]


X


array([[   0,    0,    0,    0,    0,  253,    1,    3,  292,    4,  109,
          10,   31],
       [   0,    0,    0,    0,  253,    1,    3,  292,    4,  109,   10,
          31,  684],
       [   0,    0,    0,  253,    1,    3,  292,    4,  109,   10,   31,
         684,  513],
       [   0,    0,  253,    1,    3,  292,    4,  109,   10,   31,  684,
         513,    5],
       [   0,  253,    1,    3,  292,    4,  109,   10,   31,  684,  513,
           5,  109],
       [ 253,    1,    3,  292,    4,  109,   10,   31,  684,  513,    5,
         109,   12],
       [   1,    3,  292,    4,  109,   10,   31,  684,  513,    5,  109,
          12,   41],
       [   0,    0,    0,    0,    0,  171,    1,    3,   78,    4,   17,
          11,   30],
       [   0,    0,    0,    0,  171,    1,    3,   78,    4,   17,   11,
          30,   91],
       [   0,    0,    0,  171,    1,    3,   78,    4,   17,   11,   30,
          91,    5],
       [   0,    0,  171,    1,    3,   78,    4, 


Dictionary first 10 words


{'9999': 1,
 'nosig': 2,
 'wm': 3,
 'm': 4,
 'cavok': 5,
 '12': 6,
 '11': 7,
 '13': 8,
 '10': 9,
 '16': 10}

In [3]:
#@title Train the model

from keras.layers import Bidirectional

# Build the LSTM model
model = Sequential([
    Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=20, input_length=sequence_length),
    Bidirectional(LSTM(130)),

    Dense(len(tokenizer.word_index)+1, activation='softmax')
])


# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.01), metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=70,batch_size=512)

#save the model
model.save("/content/drive/MyDrive/Colab Notebooks/airport_ml/LEVX/algorithms/LEVXml.keras")


Epoch 1/70
346/346 [==============================] - 27s 69ms/step - loss: 3.7490 - accuracy: 0.2339
Epoch 2/70
346/346 [==============================] - 24s 69ms/step - loss: 2.6668 - accuracy: 0.3287
Epoch 3/70
346/346 [==============================] - 23s 68ms/step - loss: 2.3835 - accuracy: 0.3710
Epoch 4/70
346/346 [==============================] - 24s 68ms/step - loss: 2.2300 - accuracy: 0.4012
Epoch 5/70
346/346 [==============================] - 24s 69ms/step - loss: 2.1426 - accuracy: 0.4178
Epoch 6/70
346/346 [==============================] - 23s 67ms/step - loss: 2.0747 - accuracy: 0.4286
Epoch 7/70
346/346 [==============================] - 23s 68ms/step - loss: 2.0161 - accuracy: 0.4372
Epoch 8/70
346/346 [==============================] - 23s 68ms/step - loss: 1.9566 - accuracy: 0.4477
Epoch 9/70
346/346 [==============================] - 23s 67ms/step - loss: 1.9029 - accuracy: 0.4565
Epoch 10/70
346/346 [==============================] - 23s 68ms/step - loss: 1.842